#1.Preprocesamiento de las noticias
En el siguiente segmento se realizara el recorte por tamaño de 500 caracteres por cada noticia para poder utilizar el modelo de fake news, es importante que según la longitud de cada noticia, sera igual a la cantidad de fragmentos generados por dicha noticia.

pd: inicialmente este notebook se corrio utilizando colab con GPU

In [ ]:
#Se inicializa el notebook generando una conexión con el drive donde se encuentran los CSV
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Se instala la librería de Transformers, esta es propia de Hugging face donde se pueden importar modelos mediante un Pipeline
#esta es especializada en trabajar con modelos basados en transformers, en este caso orientado a modelos de NLP
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 15.6 MB/s 
     |████████████████████████████████| 163 kB 68.7 MB/s 
     |████████████████████████████████| 7.6 MB 52.4 MB/s 


Cargamos las librerias

In [ ]:
#De la libreria transformers importamos la función del pipeline que permite obtener la estructura del modelo que deseamos
from transformers import pipeline
#Usamos tqdm la cual es una función para observar los tiempos de carga de los for
from tqdm import tqdm
#la libreria nltk nos permite utilizar funciones, especialmente las orientadas a NLP, en este caso se utiliza para decargar un complemento del modelo fakenews
import nltk
#La librería pandas nos permite leer, editar, crear y exportar dataframe donde tenemos alojados los datos o donde ponemos los datos obtenidos por el modelo
import pandas as pd
import numpy as np

In [ ]:
#Se carga la base de datos de las noticias en la variable "noticias"
noticias = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /noticias.csv')

In [ ]:
#Se imprime la noticia para observar su estructura, los datos necesarios y si cuenta con algun problema
noticias.head()

,news_id,news_url_absolute,news_init_date,news_final_date,news_title,news_text_content
0,news10006,https://www.bluradio.com/economia/precio-dolar...,2022-07-30,2022-08-14,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ..."
1,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...
2,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14,Gustavo Petro: ?Que dice el informe final de s...,El equipo de empalme del gobierno del presiden...
3,news10028,https://www.lanacion.com.ar/estados-unidos/cua...,2022-07-30,2022-08-14,Cuanto cuesta y como enviar dinero de Estados ...,?Buscas opciones para el envio de dinero de Es...
4,news10029,https://www.lanacion.com.ar/estados-unidos/com...,2022-07-30,2022-08-14,Como enviar dinero de Estados Unidos a otros p...,?Buscas opciones para el envio de dinero de Es...


In [ ]:
#Creamos un segundo DataFrame en blanco
noticias_frag= pd.DataFrame()
#Al nuevo Dataframe le abrimos un nuevo feature de nombre "noticias_total", en el cual guardamos la estrutura del titulo del texto, una separación y el contenido del texto 
noticias_frag["noticias_total"]= noticias["news_title"]+ " [SEP] "+ noticias["news_text_content"]
#De la misma forma agregamos los identificadores de la noticia para que cada "noticias_total" tenga asociado su id
noticias_frag["noticia_id"]= noticias["news_id"]
#Observamos el resultado
noticias_frag.head()

,noticias_total,noticia_id
0,Precio dolar hoy: la cotizacion de la divisa a...,news10006
1,Es cierto: El presidente Gustavo Petro quiere ...,news10011
2,Gustavo Petro: ?Que dice el informe final de s...,news10015
3,Cuanto cuesta y como enviar dinero de Estados ...,news10028
4,Como enviar dinero de Estados Unidos a otros p...,news10029


In [ ]:
#Creamos un tercer DF donde creamos los siguientes feature: id y fragmento, estos seran para guardar los fragmentos finales producidas por el ciclo
noticias_intervalo= pd.DataFrame({'id':pd.Series(dtype='str'),'fragmento':pd.Series(dtype='str')}) 

In [ ]:
#importamos numpy una libreria que nos permite manipular y utilizar funciones tanto de matemática basica como algebra lineal y vectorial
import numpy as np
#Iniciamos un primer ciclo que recorrera la longitud del dataframe, esto quiere decir que se movera noticia por noticia ejecutando los procesos internos establecidos
for j in tqdm(range(len(noticias_frag))):
#Se guarda en la variable text el contenido de la noticia en la posición j del df
    text = noticias_frag['noticias_total'][j]
#Se guaerda en la variable arr la longitud de la noticia que queremos trabajar y los intervalos (paso) en el que vamos a trabajar la noticia
    arr = np.array(range(0,len(text),500))
#Se inicializa por cada ciclo el vector divisiones_texto con el fin de que sea un contenedor de los fragmentos de texto obtenidos cada 500 caracteres
    divisiones_texto = []
#el for interno recorre la longitud de la noticia que se ha definido en el vector "arr"
    for i in range(len(arr)):
#Generamos la condición de que si la posición i es menor a la longitud de la noticia -1 nos guarde el intervalo del text entre la longitud i y la longitud i+1
        if i < len(arr)-1:
            divisiones_texto.append(text[arr[i]:arr[i+1]])
        else:
#de otro modo si es menor al primer intervalo, se guarda directamente
            divisiones_texto.append(text[arr[i]:len(text)])
#Aquí se genera un diccionario que nos permite guardar de forma ordenada los Id y las divisiones_texto, guardando cada fragmento con el id de su noticia correspondiente
    textos_divi =  {'id': noticias_frag['noticia_id'][j], 'fragmento': divisiones_texto}
#Se crea el dataframe df donde se guardan todos los datos de los fragmentos
    df = pd.DataFrame(textos_divi)
#se guardan en el dataframe noticias_intervalo, los df por noticia 
    noticias_intervalo = noticias_intervalo.append(df)

100%|██████████| 23377/23377 [08:37<00:00, 45.17it/s]


In [ ]:
#Re-inicializamos los indices 
noticias_intervalo= noticias_intervalo.reset_index(drop=True)

In [ ]:
#Guardamos el resultado en un csv como backup, para tener una muestra de los fragmentos
noticias_intervalo.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_news_frag_with_ID.csv', index=False)

#Filtro Fake News

In [ ]:
# Este paso se genera en caso de solo poder llegar hasta el paso anterior
noticias_intervalo=pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_news_frag_with_ID.csv')

In [ ]:
#Guardamos en una variable en strig con el nombre del modelo que nos interesa obtener del Pipeline
ckpt = "Narrativaai/fake-news-detection-spanish"
#Cargamos el modelo en una variable fakenews, de forma que establecemos la etiqueta del modelo, el nombre del modelo y el hiperparamentro device para saber si queremos o no que utilice GPU 1=no y 0=si
fakenews = pipeline("text-classification", model=ckpt, device=0)


Downloading:   0%|          | 0.00/898 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/858k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
#Descomprimimos el complemento punkt el cual es un tokenizador propio del modelo que es necesario para su ejecución
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Corremos el modelo-preentrenado con los valores en noticias_intervalo, en forma de lista, los valores fueron divididos en paquetes de 500
#porque el modelo solo soporta pasos de maximo 570 carácteres
fake=fakenews(list(noticias_intervalo["fragmento"].values))


In [ ]:
#guardamos el resultado (que es un conjunto de diccionarios) en formato df
fake_df= pd.DataFrame(fake)

In [ ]:
fake_df.head()

,label,score
0,REAL,0.999487
1,FAKE,0.999920
2,FAKE,0.999975
3,FAKE,0.999973
4,FAKE,0.999969


In [ ]:
#Asignamos los respectivos id de noticia con cada uno de los labels y scores generados del modelo
fake_df['id'] = noticias_intervalo['id']

In [ ]:
#guardamos el resultado obtenido en un csv
fake_df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_labels_complete_score_with_ID_partes.csv', index=False)

In [ ]:
fake_df.head()

,label,score,id
0,REAL,0.999487,news10006
1,FAKE,0.999920,news10006
2,FAKE,0.999975,news10011
3,FAKE,0.999973,news10011
4,FAKE,0.999969,news10011


In [ ]:
#en esta funcion promediamos las probabilidades de los diferentes fragmentos de texto con el fin de poder encontrar si es mas probable que sea Real o Fake
labels = []
for i in tqdm(fake_df['id'].unique()):
  df2 = fake_df[fake_df['id']== i].reset_index()
  real =[]
  fake = []
  for j in range(len(df2)):
    if df2['label'][j] == 'REAL':
      real.append(df2['score'][j])
      fake.append(1-df2['score'][j])
    else:
      fake.append(df2['score'][j])
      real.append(1-df2['score'][j])
  if np.mean(real) > np.mean(fake): 
    labels.append('REAL')
  else:
    labels.append('FAKE')

100%|██████████| 23377/23377 [11:21<00:00, 34.29it/s]


In [ ]:
#convertimos lo anterior en un dataframe en el cual vamos a tener el label definitivo del detector de fake news para cada noticia
d = {'id':fake_df['id'].unique(),'Label':labels}
df = pd.DataFrame(d)
df.tail()

,id,Label
23372,news99983,REAL
23373,news99989,FAKE
23374,news99991,FAKE
23375,news99996,FAKE
23376,news99997,FAKE


In [ ]:
#guardamos este resultado en un csv
df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_definitivo_with_id.csv', index=False)

#Clasificador multiple

In [ ]:
#Se descarga el clasificador, usando el pipeline zeroshot, se recomiendo poner el hiperparametro device=0 para utilizar GPU
classifier = pipeline("zero-shot-classification", 
                       model="Recognai/bert-base-spanish-wwm-cased-xnli", device=0)
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

#Obtener sinonimos de base actualizable

In [ ]:
#Se carga el csv de sinonimos
sinonimos = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Sinonimos_Caracteristicas  - Sinonimos_Caracteristicas .csv')

In [ ]:
sinonimos

,Caracteristica,sinonimo
0,sostenibilidad,sostenibilidad
1,sostenibilidad,viabilidad
2,sostenibilidad,factibilidad
3,sostenibilidad,sustentabilidad
4,sostenibilidad,permanencia
...,...,...
98,otros,violencia
99,otros,literatura
100,otros,religión
101,otros,secuestro


In [ ]:
#Se guardan los sinonimos en variables separadas con formado dataframe  
Sos=sinonimos[(sinonimos.Caracteristica=="sostenibilidad")]
Ali=sinonimos[(sinonimos.Caracteristica=="alianza")]
Rep=sinonimos[(sinonimos.Caracteristica=="reputación")]
Reg=sinonimos[(sinonimos.Caracteristica=="regulación")]
Des=sinonimos[(sinonimos.Caracteristica=="desarrollo")]
Inn=sinonimos[(sinonimos.Caracteristica=="innovación")]
Mac=sinonimos[(sinonimos.Caracteristica=="macroeconomía")]
Otr=sinonimos[(sinonimos.Caracteristica=="otros")]

In [ ]:
#Los df son convertidos en vectores y almacenados en variables para ser utilizados en el modelo
sos=Sos['sinonimo'].to_numpy()
ali=Ali['sinonimo'].to_numpy()
rep=Rep['sinonimo'].to_numpy()
reg=Reg['sinonimo'].to_numpy()
des=Des['sinonimo'].to_numpy()
inn=Inn['sinonimo'].to_numpy()
mac=Mac['sinonimo'].to_numpy()
otr=Otr['sinonimo'].to_numpy()

In [ ]:
#Se agrupan los vectores de sinonimos en una sola variable candidate_labels
candidate_labels = [sos, ali, rep, reg, des, inn, mac, otr]
#Se genera la clasificación guardando los resultados en la variable Pred_L de forma que lo obtenido son dic con dos features labels y score
Pred_L = classifier(list(noticias['news_text_content'].values), candidate_labels, multi_labels=True)

In [ ]:
# se convierte el resultado en un csv
label_df = pd.DataFrame(Pred_L)

In [ ]:
#se asigna el id de la noticia a cada resultado
label_df['id'] = noticias['news_id']

In [ ]:
#guardamos los resultados del modelo en un csv
label_df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorias.csv',index= False )

In [ ]:
label_df = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorias.csv')
label_df.head()

,labels,scores,id,categoria
0,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.2376488447189331, 0.219680055975914, 0.1971...",news10006,reputación
1,"[array(['reglamentación', 'organización', 'reg...","[0.41854286193847656, 0.19385991990566254, 0.1...",news10011,regulación
2,"[array(['reglamentación', 'organización', 'reg...","[0.29956182837486267, 0.17904965579509735, 0.1...",news10015,regulación
3,"[array(['reglamentación', 'organización', 'reg...","[0.26417383551597595, 0.19620046019554138, 0.1...",news10028,regulación
4,"[array(['reglamentación', 'organización', 'reg...","[0.28489089012145996, 0.19738508760929108, 0.1...",news10029,regulación


In [ ]:
# en esta celda se comprueba cual es el vector de sinonimos que posee la probabilidad mas alta con el fin de poder asignar las categorias 
#(la probabilidad mas alta siempre es el primer arreglo de scores y labels)
categoria =[]
for i in range(len(label_df)):
  if np.array_equal(label_df['labels'][i][0],sos):
    categoria.append("sostenibilidad")
  if np.array_equal(label_df['labels'][i][0],ali):
    categoria.append("alianza")
  if np.array_equal(label_df['labels'][i][0],rep):
    categoria.append("reputación")
  if np.array_equal(label_df['labels'][i][0],reg):
    categoria.append("regulación")
  if np.array_equal(label_df['labels'][i][0],des):
    categoria.append("desarrollo")
  if np.array_equal(label_df['labels'][i][0],inn):
    categoria.append("innovación")
  if np.array_equal(label_df['labels'][i][0],mac):
    categoria.append("macroeconomía")
  if np.array_equal(label_df['labels'][i][0],otr):
    categoria.append("otros")

In [ ]:
#Agregamos la categoria al df
label_df['categoria'] = categoria
label_df.head()

In [ ]:
#guardamos el resultado
label_df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorias.csv',index= False )

#Proceso para filtrar la categoria final
Teniendo los resultados filtrados por noticia tanto de fake news como de el zero-shot para la categorizacion solo debemos comprobar si es o no una fake news para agregarlo en la categoria descarable o asignar la categoria correspondiente

In [ ]:
# se unen los dos dataframes por su id
df_definitive = df.merge(label_df,on='id')
df_definitive.head()

,id,Label,labels,scores,categoria
0,news10006,FAKE,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.2376488447189331, 0.219680055975914, 0.1971...",reputación
1,news10011,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.41854286193847656, 0.19385991990566254, 0.1...",regulación
2,news10015,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.29956182837486267, 0.17904965579509735, 0.1...",regulación
3,news10028,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.26417383551597595, 0.19620046019554138, 0.1...",regulación
4,news10029,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.28489089012145996, 0.19738508760929108, 0.1...",regulación


In [ ]:
#se compruba si la noticia es falsa o se asigna la categoria de no serlo
true_cat =[]
for i in range(len(df_definitive)):
  if df_definitive['Label'][i] == 'FAKE':
    true_cat.append('descartable')
  else:
    true_cat.append(df_definitive['categoria'][i])

In [ ]:
# se agrega la categoria final al dataframe
df_definitive['True_cat'] = true_cat

De aqui en adelante se realizan ciertas modificaciona al dataframe con el fin de cumplir con el formato del archivo categorizacion.csv de los entregables

In [ ]:
df_definitive.rename(columns = {'id':'news_id'}, inplace = True)

In [ ]:
df_definitive.head()

,news_id,Label,labels,scores,categoria,True_cat
0,news10006,FAKE,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.2376488447189331, 0.219680055975914, 0.1971...",reputación,descartable
1,news10011,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.41854286193847656, 0.19385991990566254, 0.1...",regulación,descartable
2,news10015,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.29956182837486267, 0.17904965579509735, 0.1...",regulación,descartable
3,news10028,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.26417383551597595, 0.19620046019554138, 0.1...",regulación,descartable
4,news10029,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.28489089012145996, 0.19738508760929108, 0.1...",regulación,descartable


In [ ]:
df_definitive['nombre_equipo']=  ['Goofy_Goobers' for x in range(len(df_definitive))]
df_definitive.head()

,news_id,Label,labels,scores,categoria,True_cat,nombre_equipo
0,news10006,FAKE,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.2376488447189331, 0.219680055975914, 0.1971...",reputación,descartable,Goofy_Goobers
1,news10011,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.41854286193847656, 0.19385991990566254, 0.1...",regulación,descartable,Goofy_Goobers
2,news10015,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.29956182837486267, 0.17904965579509735, 0.1...",regulación,descartable,Goofy_Goobers
3,news10028,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.26417383551597595, 0.19620046019554138, 0.1...",regulación,descartable,Goofy_Goobers
4,news10029,FAKE,"[array(['reglamentación', 'organización', 'reg...","[0.28489089012145996, 0.19738508760929108, 0.1...",regulación,descartable,Goofy_Goobers


In [ ]:
df_definitive = df_definitive.drop(columns=['Label','labels','scores','categoria'])

In [ ]:
df_definitive.head()

,news_id,True_cat,nombre_equipo
0,news10006,descartable,Goofy_Goobers
1,news10011,descartable,Goofy_Goobers
2,news10015,descartable,Goofy_Goobers
3,news10028,descartable,Goofy_Goobers
4,news10029,descartable,Goofy_Goobers


In [ ]:
df_definitive.rename(columns = {'True_cat':'categoria'}, inplace = True)

In [ ]:
df_definitive.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorizacion.csv',index= False )